In [1]:
# подгружаемые библиотеки
import numpy as np
import tensorflow as tf
import re

from bpemb import BPEmb

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
vocab_size = 3000
emb_dim = 100
max_len = 200 # TODO посчитать максимальную длинну выборки
buffer_size = 101# размер буфера данных в батчах

In [3]:
bpemb_ru = BPEmb(lang='ru', vs=vocab_size , dim=emb_dim)

# bpemb_ru.encode_ids('.,!?:;')
#>>> [2898, 679, 2978, 2985, 2947, 2963]
def load_emb_matrix(bpemb= bpemb_ru, dtype=np.float32):
    return bpemb.emb.vectors.astype(dtype)

def parse_fn(line, bpemb = bpemb_ru):

    sequence = np.array(bpemb.encode_ids(line)).astype(np.int32)

    feature = np.hstack(([1],sequence[:-1]))
    
    labels = ( #TODO изменить кастыли с циферками 2898-, 679-. 2978-! 2985-? 2947-: 2963-; 
              (sequence == 2898)*1 + 
              (sequence == 679)*2 + 
              (sequence == 2978)*3 + 
              (sequence == 2985)*4 + 
              (sequence == 2947)*5 + 
              (sequence == 2963)*6
             ).astype(np.int32) 
    
    mask = ((labels == 0)*1 + (labels != 0)).astype(np.float32)
    
    return (feature, len(feature)), (labels, len(labels), mask)


def generator_fn(data_file_url):
    with open(data_file_url, 'r') as file:
        for row in file:
            yield parse_fn(row[:-1])


def input_fn(data_file_url, params, mode):

    shapes = (([None], ()), ([None], (), [None]))
    types = ((tf.int32, tf.int32), (tf.int32, tf.int32, tf.float32))
    defaults = ((1, 0), (1, 0, 0.))

    dataset = tf.data.Dataset.from_generator(lambda : generator_fn(data_file_url),
                                             output_shapes=shapes, output_types=types)
    
    if mode == 'train':
        dataset = dataset.shuffle(buffer_size=params['train_size'], reshuffle_each_iteration=True)
        dataset = dataset.repeat(params['num_epochs'])
    
    
    dataset = dataset.repeat(params['num_epochs'])
    return (dataset.padded_batch(params['batch_size'], shapes).prefetch(1))

In [4]:
# модель
def model_fn(features, labels, mode, params):
    
    sequences, lengths = features
    current_butch_size = tf.shape(sequences)[0]

    # матрица эмбеддингов decoder-а
    embeddings = tf.Variable(initial_value = load_emb_matrix() ,trainable=False)
    
    # decoder
    cell  = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.GRUCell(num_units=params['lstm_hidden_dim']) 
                          for _ in range(params['num_layers'])])
    
    # началное состояние
    initial_state = tuple([tf.tile(tf.constant([[0]], tf.float32), [current_butch_size, params['lstm_hidden_dim']]) 
                           for _ in range(params['num_layers'])]) 

        

    sequences_embedded = tf.nn.embedding_lookup(params=embeddings, ids=sequences)

    train_helper = tf.contrib.seq2seq.TrainingHelper(inputs=sequences_embedded, 
                                                      sequence_length=lengths)

    decoder = tf.contrib.seq2seq.BasicDecoder(cell=cell, 
                                              helper=train_helper, 
                                              initial_state=initial_state
                                             )

    outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder=decoder,
                                                      maximum_iterations=params['max_iter'], 
                                                      impute_finished=True,
                                                      )

    
    training = mode == tf.estimator.ModeKeys.TRAIN
    dropout = tf.layers.dropout(inputs=outputs.rnn_output, 
                                rate= 1 - params['dropout_rate'], 
                                training=training)

    dense = tf.layers.dense(dropout, params['output_vocab_size'])
    

    logits = dense
    sample_id = tf.argmax(logits, 2)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'sequences': sequences,
            'prediction': sample_id,
            'lengths': lengths
        }

        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    else:
        target, target_lengths, mask = labels

        #mask = tf.sequence_mask(lengths, dtype=tf.float32)

        metrics = {
            'acc': tf.metrics.accuracy(target, sample_id, mask),
            #'f1_score' : tf.contrib.metrics.f1_score(target, sample_id, mask),
        }

        loss = tf.contrib.seq2seq.sequence_loss(logits=logits, 
                                                targets=target, 
                                                weights=mask, 
                                                average_across_timesteps=True, 
                                                average_across_batch=True)

        # в режиме eval возвращаем усреднённый лосс
        if mode == tf.estimator.ModeKeys.EVAL:
            return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

        # в режиме train ещё и обновляем обучаемые параметры
        elif mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.AdamOptimizer
            optimizer = optimizer(learning_rate=params['learning_rate'])
            train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

            return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
    
    


In [5]:
params = {
    'vocab_size': vocab_size,
    'output_vocab_size': 7,
    'train_size': 256*buffer_size,
    'num_layers': 2,
    'embedding_dim': emb_dim,
    'dropout_rate': 0.2,
    'lstm_hidden_dim': 300,
    'max_iter': max_len,
    'batch_size': 256,
    'num_epochs': 1,
    'learning_rate': 1e-3
}

config = tf.estimator.RunConfig(model_dir='project1',
                                save_checkpoints_steps = 200,
                               save_checkpoints_secs = None)
model = tf.estimator.Estimator(model_fn=model_fn, params=params, config=config)

INFO:tensorflow:Using config: {'_model_dir': 'project1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f525680d0f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
model.train(lambda: input_fn('flibusta_full_train.txt', params=params, mode='train'))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use `argmax` instead
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into project1/model.ckpt.
INFO:tensorflow:loss = 1.940601, step = 1
INFO:tensorflow:global_step/sec: 1.06829
INFO:tensorflow:loss = 0.18132442, step = 101 (93.609 sec)
INFO:tensorflow:Saving checkpoints for 200 into project1/model.ckpt.
INFO:tensorflow:global_step/sec: 1.03941
INFO:tensorflow:loss = 0.1616047, step = 201 (96.208 sec)
INFO:tensorflow:global_step/sec: 1.0702
INFO:tensorflow:loss = 0.13889553, step = 301 (93.441 sec)
INFO:tensorflow:Saving checkpoints for 400 into project1/model.ckpt.
INFO:tensorflow:global_step/sec: 1.05828
INFO:tensorflow:loss = 0.14392494, step = 401 (94.493 sec)


In [7]:
# обучение и валидация
eval_result = model.evaluate(lambda: input_fn('eval.txt', params=params, mode='eval'))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-12-16:01:46
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from project1/model.ckpt-1800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-12-16:01:46
INFO:tensorflow:Saving dict for global step 1800: acc = 0.6312711, global_step = 1800, loss = 1.1486834
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1800: project1/model.ckpt-1800


In [18]:
# дешифровка 
d = {1:2898, 2:679, 3:2978, 4:2985, 5:2947, 6:2963}
for x in model.predict(lambda: input_fn('test.txt', params=params, mode='predicted')):
    a = []
    for i in range(x['lengths']):
        a.append(x['sequences'][i])
        if x['prediction'][i] != 0:
            a.append(d[x['prediction'][i]])
    print(bpemb_ru.decode_ids(x['sequences'][:x['lengths']]))
    print(bpemb_ru.decode_ids(np.array(a)))


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from project1/model.ckpt-1800
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
в середине разговора про политические действия анна павловна разгорячи
в середине разговора? про политические? действия? анна? павлов?на? разгорячи
анна павловна почти закрыла глаза в знак того что ни она ни кто другой не могут судить про то что угодно или нравится императри
анна! павлов!на? почти закрыла? глаза? в знак? того что? ни она ни кто друго?й не могут? судить? про то что? угодно или нрави?тся? импера?три?
князь наклонился в знак уважения и признатель
князь! наклони!лся! в знак! уважения! и признатель
и она улыбнулась своею восторженною улыб
и она? улыб?ну?лась? сво?е?ю? восто?р?жен?но?ю? улы?б?
князь не отвечал но она молча значительно глядя на него ждала ответа князь василий поморщи
князь! не отвечал! но она молча значи

In [17]:
bpemb_ru.decode_ids(np.array(a))

'князь! ипполит! перенес ей! риди!к!ю!ль перешел за нею! и близко придвинув к ней кресло сел подле не'

In [99]:
bpemb_ru.decode_ids([1,  264,  436, 2906,  333,   23, 2908, 1100,  376,  101, 1678,
       2168, 2920,   15,  152, 2903,    8,   53,  293, 2918,  147,  225])

'виконт хотел уже начать свой рассказ и тонко улыбнулся'